In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
url = "https://raw.githubusercontent.com/ilya-lysenko-An/Pet-EDA/main/london_merged.csv"
df = pd.read_csv(url)
print("Размер датафрейма:", df.shape)

In [ ]:
print("проверка nbstripout")